In [13]:
using CSV
using DataFrames
using XLSX
using Statistics
using Glob
using MLJ
using MLJBase
using DelimitedFiles
using CategoricalArrays

In [14]:
response_name = ["no_tuber", "no_tuber_scab", "no_tuber_scabpit"
    , "no_tuber_scabsuper", "scab_severity", "yield_per_meter", "yield_per_plant"]
for i in 1:7
    write_to_xlsx(i, response_name[i])
end

In [9]:
function write_to_xlsx(response, name)
    data = get_all_level(response)
    filename = "../processed-data/result-file/$name.xlsx"
    labels = collect(0:1:19)
    XLSX.openxlsx(filename, mode="w") do xf
        sheet = xf[1]
        XLSX.rename!(sheet, "Phylum")
        XLSX.writetable!(sheet, data[1], vcat("", labels))
        XLSX.addsheet!(xf)
        XLSX.rename!(xf[2], "Class")
        XLSX.writetable!(xf[2], data[2], vcat("", labels))
        XLSX.addsheet!(xf)
        XLSX.rename!(xf[3], "Order")
        XLSX.writetable!(xf[3], data[3], vcat("", labels))
    end
end

write_to_xlsx (generic function with 1 method)

In [5]:
function get_all_level(response)
    all_level = ["Phylum", "Class", "Order"]
    all = Any[]
    for i in 1:length(all_level)
        level = all_level[i]
        path = "../processed-data/$level/result"
        filename = glob("*$response.txt", path)
        data = single_response(filename)     
        push!(all, data)
    end    
    return all
end

get_all_level (generic function with 1 method)

In [6]:
function single_response(filename)
    all = Any[]
    row_num = [0,1,2,3,4]
    push!(all, row_num)
    for i in 1:length(filename)
        data = readdlm(filename[i])
        new = calculation(data)
        push!(all, new)
    end
    return all
end

single_response (generic function with 1 method)

In [7]:
function calculation(data)
    ret = zeros(5)
    # set prediction and truth
    truth = categorical(data[:, 2])
    predict = categorical(data[:, 4])
    measures("FScore")
    # F for zero
    m = FScore(rev=true)
    ret[1] = m(predict, truth)
    # F for one
    m = FScore(rev=false)
    ret[2] = m(predict, truth)
    ret[3] = accuracy(predict, truth)
    m = MulticlassFScore(average = MLJBase.MacroAvg())
    ret[4] = m(predict, truth)
    p = [(i, count(==(i), truth)) for i in unique(truth)]
    ret[5] = (p[1][2]*ret[2] + p[2][2]*ret[1])/(p[1][2] + p[2][2])
    return ret
end

calculation (generic function with 1 method)